<a href="https://colab.research.google.com/github/Pamela120609/barchart-udt/blob/master/Projeto_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import json # Para trabalhar com a resposta JSON do Gemini
from datetime import datetime, timedelta # Para ajudar com datas e cálculos de tempo
import google.colab.userdata # Para acessar a chave de API de forma segura
import textwrap # Para formatar o texto de saída

# Função auxiliar para formatar a saída do texto
def to_markdown(text):
  text = text.replace('```json', '').replace('```', '').strip()
  # Adiciona identação para destacar a resposta do bot
  return textwrap.indent(text, '> ', predicate=lambda line: True)


# --- Estrutura dos Itens (Agenda, Tarefas, Aniversários) ---

agenda_items = []

def adicionar_item(descricao, data, hora, categoria="Pessoal", tipo_item="Evento"):
    """
    Adiciona um novo item à lista local.
    Espera data no formatoYYYY-MM-DD e hora no formato HH:MM (opcional).
    Inclui agora o tipo de item (Evento, Tarefa, Aniversário, Estudo IA).
    """
    novo_item = {
        "descricao": descricao,
        "data": data,
        "hora": hora,
        "categoria": categoria,
        "tipo_item": tipo_item
    }
    agenda_items.append(novo_item)
    # Mensagem de confirmação ajustada - sem mencionar Agenda do Google
    print(f"✨ Feito! Adicionei o {tipo_item} '{descricao}' para você. Pode deixar comigo! 😉")

def listar_itens():
    """
    Lista todos os itens, ordenados por data e hora.
    Inclui agora o tipo de item na listagem.
    """
    if not agenda_items:
        print("📅 Sua lista de itens está vazia. Que tranquilidade! 😊")
    else:
        print("--- Dando uma olhada nos seus próximos itens... ---")
        try:
            agenda_items.sort(key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))
        except Exception as e:
            print(f"Aviso: Não consegui ordenar seus itens direito, desculpe! Erro: {e} ")

        for i, item in enumerate(agenda_items):
            descricao = item.get('descricao', 'Sem descrição')
            data = item.get('data', 'Sem data')
            hora = item.get('hora', 'Sem hora')
            categoria = item.get('categoria', 'Sem categoria')
            tipo_item = item.get('tipo_item', 'Item')
            print(f"{i+1}. [{tipo_item}] Descrição: {descricao}, Data: {data}, Hora: {hora}, Categoria: {categoria}")
        print("------------------------------------------------")

# --- Configuração da API do Google Gemini ---

!pip install -q -U google-generativeai
import google.generativeai as genai
API_KEY = None
model = None # Inicializa model como None também

# Tenta configurar a API e carregar o modelo
try:
    API_KEY = google.colab.userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        raise ValueError("Chave de API não encontrada ou está vazia nos Secrets do Colab.")

    # Configura a biblioteca do Gemini com a chave encontrada
    genai.configure(api_key=API_KEY)
    print("Biblioteca google-generativeai instalada e API configurada. ✨ Prontos para conversar! 😉")


    model_name = 'gemini-2.0-flash' # <-- Usando o modelo 2.0 Flash
    model = genai.GenerativeModel(model_name)
    print(f"Modelo '{model_name}' carregado com sucesso. 🎉 Vamos lá!")

except Exception as e:
    print(f"ERRO crítico na configuração da API ou carregamento do modelo: {e} 😭 Que chato!")
    print("Parece que tivemos um probleminha para conectar com a inteligência.")
    print("Por favor, verifique se sua chave de API está correta e se o acesso está liberado.")
    print(f"Detalhes técnicos: {e}")
    model = None # Garante que model é None se houver erro

# --- Funções de Análise, Sugestão Criativa e Bem-Estar por Humor (Usando Gemini com Persona) ---

def analisar_agenda_criativo(agenda, periodo="próximos itens"):
    """
    Usa o Gemini para analisar a lista de itens e gerar um resumo criativo ou sugestões,
    agindo como um assistente pessoal humano.
    Simula um "Agente Analista".
    """
    if not agenda:
        print(f"Sua lista de itens está vazia. Nada para analisar {periodo}. Que tal adicionar algo? 😊")
        return

    if model is None:
         print("Não consigo analisar seus itens agora, a inteligência não está conectada. 😔")
         return

    # Formata a agenda para o prompt
    agenda_ordenada = sorted(agenda, key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))

    # Limita a agenda apresentada ao Gemini (ex: próximos 10 itens ou 7 dias)
    hoje = datetime.now().date()
    agenda_relevante = []
    for item in agenda_ordenada:
        item_data_str = item.get('data', '')
        if item_data_str:
            try:
                item_data = datetime.strptime(item_data_str, '%Y-%m-%d').date()
                if item_data >= hoje:
                     agenda_relevante.append(item)
            except ValueError:
                 pass

    agenda_para_prompt = agenda_relevante[:10] # Limita aos próximos 10 itens

    if not agenda_para_prompt:
         print(f"Não há itens futuros próximos para analisar {periodo}. 😊 Tudo livre!")
         return

    # Formata a lista de itens relevante para o prompt
    agenda_formatada = "Próximos itens:\n"
    for item in agenda_para_prompt:
         descricao = item.get('descricao', 'Sem descrição')
         data = item.get('data', 'Sem data')
         hora = item.get('hora', 'Sem hora')
         tipo_item = item.get('tipo_item', 'Item')
         agenda_formatada += f"- [{tipo_item}] {data} {hora}: {descricao}\n"


    # ** PROMPT PARA ANÁLISE CRIATIVA COM PERSONA HUMANA **
    prompt_analise = f"""
Você é um assistente pessoal amigável, proativo e com um leve senso de humor. Sua tarefa é analisar a seguinte lista de próximos itens do usuário (eventos, tarefas, aniversários) e gerar um pequeno resumo criativo ou uma observação interessante sobre como o dia/período parece.

Considere a distribuição dos itens, os tipos (Eventos, Tarefas, Aniversários) e os horários. Use emojis para tornar a análise mais amigável e visual. Sugira algo proativo, como um momento para relaxar, focar, ou se preparar para algo. Fale como se estivesse conversando diretamente com o usuário, de forma natural e com um toque de personalidade.

Lista de Itens:
{agenda_formatada}

Gere a análise criativa (seja humano e amigável!):
"""
    try:
        response = model.generate_content(prompt_analise, generation_config=genai.GenerationConfig(temperature=0.8)) # Temperatura mais alta para criatividade e personalidade
        print(f"\n--- Dando uma olhada nos seus itens ({periodo})... ---")
        print(to_markdown(response.text))
        print("---------------------------------------------------")
    except Exception as e:
        print(f"Ocorreu um erro ao analisar seus itens. Que pena! {e} 😔")
        print("Não foi possível gerar a análise no momento.")

def sugerir_pausa_inteligente(agenda):
    """
    Usa o Gemini para analisar a lista de itens e sugerir onde pausas podem ser úteis,
    agindo como um assistente pessoal humano.
    """
    # Esta função reutiliza a análise, focando no contexto de pausas,
    # mas a resposta virá com a persona definida no prompt de análise.
    analisar_agenda_criativo(agenda, periodo="para sugerir pausas")

def renomear_item_criativo(descricao_original):
    """
    Usa o Gemini para sugerir nomes criativos para um item,
    agindo como um assistente pessoal humano.
    """
    if model is None:
        print("Não consigo sugerir nomes criativos agora, a inteligência não está conectada. 😔")
        return

    # ** PROMPT PARA RENOMEAR CRIATIVAMENTE COM PERSONA HUMANA **
    prompt_renomear = f"""
Você é um assistente pessoal amigável e com um leve senso de humor. Sua tarefa é sugerir 3-5 nomes criativos, divertidos ou interessantes para um item com a seguinte descrição:

Descrição original: "{descricao_original}"

Sugira nomes criativos. Use emojis. Liste as sugestões claramente. Fale de forma natural e com um toque de humor.

Exemplo:
Descrição: "Reunião com chefe"
Sugestões (com humor):
- 🗣️ Encontro Estratégico com o Líder Supremo (prepare-se!)
- ☕️ Papo com o Chefe (tomara que tenha café!)
- 📊 Alinhamento de Planetas (se for importante... ou caótico!)
"""
    try:
        response = model.generate_content(prompt_renomear, generation_config=genai.GenerationConfig(temperature=0.9)) # Temperatura alta para criatividade e humor
        print(f"\n--- Pensando em nomes divertidos para '{descricao_original}'... 🤔 ---")
        print(to_markdown(response.text))
        print("-----------------------------------------------------------------")
    except Exception as e:
        print(f"Ocorreu um erro ao sugerir nomes. Que pena! {e} 😔")
        print("Não foi possível gerar sugestões de nomes no momento.")

# --- Função: Sugerir Bem-Estar Baseado no Humor ---

def sugerir_bem_estar_com_humor(humor, tipo_sugestao):
    """
    Usa o Gemini para sugerir atividades/conteúdo de bem-estar
    baseado no humor e tipo de sugestão desejada pelo usuário,
    agindo como um assistente pessoal humano.
    Simula um "Agente Bem-Estar Personalizado".
    """
    if model is None:
        print("Não consigo sugerir atividades agora, a inteligência não está conectada. 😔")
        return

    # ** PROMPT PARA SUGESTÃO DE BEM-ESTAR COM HUMOR E PERSONA HUMANA **
    prompt_bem_estar = f"""
Você é um assistente pessoal amigável, empático e com um leve senso de humor, focado em bem-estar. O usuário está se sentindo "{humor}" e pediu sugestões de "{tipo_sugestao}".

Sua tarefa é sugerir 3-5 atividades relaxantes, filmes, livros ou comidas (dependendo do tipo de sugestão) que combinem com o humor "{humor}". Seja criativo, empático e use um tom humano e amigável. Use emojis relevantes.

Sugestões para quem está "{humor}" ({tipo_sugestao}):
"""
    try:
        response = model.generate_content(prompt_bem_estar, generation_config=genai.GenerationConfig(temperature=0.9)) # Temperatura alta para criatividade e empatia
        print(f"\n--- Pensando em algo bom para você, já que está se sentindo {humor}... 😊 ---")
        print(to_markdown(response.text))
        print("--------------------------------------------------------------------------")
    except Exception as e:
        print(f"Ocorreu um erro ao sugerir atividades. Que pena! {e} 😔")
        print("Não foi possível gerar sugestões de bem-estar no momento.")

# --- NOVA FUNÇÃO: Sugerir Horário para Estudar IA ---

def sugerir_horario_estudo_ia(agenda):
    """
    Orquestra a sugestão de horário para estudar IA.
    Pergunta ao usuário sobre preferências, analisa a agenda com Gemini,
    sugere o horário e pergunta se quer adicionar à lista.
    Simula um "Agente de Produtividade e Estudo".
    """
    if model is None:
        print("Não consigo sugerir horários de estudo agora, a inteligência não está conectada. 😔")
        return

    print("\nOk, vamos encontrar o melhor momento para você mergulhar nos estudos de IA! 🧠")

    # 1. Perguntar ao usuário sobre preferências
    periodo_produtivo = input("Primeiro, me diga: qual período do dia você se sente mais produtivo? (Manhã, Tarde, Noite): ").strip()
    topico_estudo = input("E sobre o que especificamente em IA você quer estudar? (Ex: Machine Learning, Redes Neurais, Prompt Engineering): ").strip()

    if not periodo_produtivo or not topico_estudo:
        print("Hmm, preciso saber seu período produtivo e o tópico para te ajudar melhor. 😉")
        return

    # 2. Formatar a agenda para o prompt do Gemini
    agenda_ordenada = sorted(agenda, key=lambda x: (str(x.get('data', '')), str(x.get('hora', ''))))
    hoje = datetime.now().date()
    agenda_relevante = []
    for item in agenda_ordenada:
        item_data_str = item.get('data', '')
        if item_data_str:
            try:
                item_data = datetime.strptime(item_data_str, '%Y-%m-%d').date()
                if item_data >= hoje and item_data <= (hoje + timedelta(days=7)): # Considera próximos 7 dias
                     agenda_relevante.append(item)
            except ValueError:
                 pass

    agenda_formatada = "Próximos itens nos próximos 7 dias:\n"
    if not agenda_relevante:
        agenda_formatada += "Nenhum item agendado.\n"
    else:
        for item in agenda_relevante:
            descricao = item.get('descricao', 'Sem descrição')
            data = item.get('data', 'Sem data')
            hora = item.get('hora', 'Sem hora')
            tipo_item = item.get('tipo_item', 'Item')
            agenda_formatada += f"- [{tipo_item}] {data} {hora}: {descricao}\n"

    # 3. Prompt para o Gemini Sugerir Horário
    prompt_sugestao_estudo = f"""
Você é um assistente pessoal amigável, proativo e inteligente, focado em ajudar o usuário a encontrar tempo para estudar. Analise a seguinte lista de próximos itens do usuário (eventos, tarefas, aniversários) nos próximos 7 dias.

Lista de Itens:
{agenda_formatada}

O usuário se sente mais produtivo no período da **{periodo_produtivo}** e quer estudar sobre **{topico_estudo}**.

Sua tarefa é:
1. Analisar a lista de itens e identificar um ou mais blocos de tempo livre nos próximos 7 dias, dando preferência ao período da **{periodo_produtivo}**.
2. Sugerir o **melhor dia e horário** para o usuário estudar sobre **{topico_estudo}**, considerando a agenda e o período produtivo informado.
3. Gere a sugestão em formato JSON, incluindo a data e hora sugeridas, e uma mensagem amigável.

Sua resposta DEVE ser APENAS um objeto JSON. Não inclua nenhum texto adicional antes ou depois do JSON. Não adicione explicações.

Exemplo de Resposta JSON:
```json
{{
  "sugestao_data": "YYYY-MM-DD (data sugerida)",
  "sugestao_hora": "HH:MM (hora sugerida)",
  "mensagem": "Com base na sua agenda e no seu período produtivo na {periodo_produtivo}, sugiro estudar sobre {topico_estudo} no dia {DD/MM} às {HH:MM}. O que acha?"
}}
```

Agora, gere a sugestão para o usuário:
"""
    try:
        print("Analisando sua agenda e preferências para encontrar o momento ideal... 🕵️‍♂️")
        response = model.generate_content(prompt_sugestao_estudo, generation_config=genai.GenerationConfig(temperature=0.5)) # Temperatura moderada para sugestão estruturada mas natural

        json_text = response.text.strip()
        # Limpeza mais robusta de marcadores de código
        while json_text.startswith("```") or json_text.endswith("```"):
             if json_text.startswith("```json"):
                  json_text = json_text[7:].strip()
             elif json_text.startswith("```"):
                  json_text = json_text[3:].strip()
             if json_text.endswith("```"):
                  json_text = json_text[:-3].strip()

        sugestao_ia = json.loads(json_text)

        sugestao_data = sugestao_ia.get('sugestao_data')
        sugestao_hora = sugestao_ia.get('sugestao_hora')
        mensagem_sugestao = sugestao_ia.get('mensagem', "Encontrei um possível horário para você estudar.")

        if sugestao_data and sugestao_hora:
            # 4. Apresentar a sugestão e perguntar se quer adicionar
            print(f"\n--- Sugestão de Horário para Estudo de IA ---")
            print(to_markdown(mensagem_sugestao))
            print(f"Sugestão: Dia {sugestao_data} às {sugestao_hora}")
            print("---------------------------------------------")

            confirmar = input("Quer adicionar esta tarefa 'Estudar IA' na sua lista? (sim/não): ").strip().lower()

            # 5. Lidar com a confirmação
            if confirmar == 'sim':
                adicionar_item(f"Estudar IA: {topico_estudo}", sugestao_data, sugestao_hora, categoria="Estudo", tipo_item="Estudo IA")
                print("Tarefa de estudo adicionada! 💪 Boa sorte!")
            elif confirmar == 'não' or confirmar == 'nao':
                print("Ok, sem problemas! 😉")
                # 6. Perguntar se tem mais alguma alteração/sugestão (interpretação: quer tentar outro horário?)
                tentar_novamente = input("Quer tentar encontrar outro horário para estudar IA? (sim/não): ").strip().lower()
                if tentar_novamente == 'sim':
                    sugerir_horario_estudo_ia(agenda) # Chama a função novamente para tentar outro horário
                else:
                    print("Tudo bem! Me diga se precisar de mais alguma coisa. 😊")
            else:
                print("Não entendi sua resposta. Vou deixar a sugestão anotada, caso mude de ideia! 😊")
        else:
             print("Desculpe, não consegui encontrar um horário ideal com base na sua agenda e preferências. 😔 Talvez sua agenda esteja muito cheia ou as preferências sejam restritivas.")

    except json.JSONDecodeError:
        print("Erro: Gemini não retornou um JSON válido ao sugerir horário de estudo. 😞")
        # print(f"Resposta bruta do Gemini: {response.text}") # Descomente para ver a resposta bruta se houver erro
        print("Não foi possível gerar a sugestão de horário no momento.")
    except Exception as e:
        print(f"Ocorreu um erro ao sugerir horário de estudo: {e} 😭")
        print("Não foi possível gerar a sugestão de horário no momento.")


# --- Função para Entender o Comando do Usuário com Gemini ---

def entender_comando_com_gemini(comando_usuario):
    """
    Usa o Gemini para analisar o comando do usuário, identificar a intenção
    e extrair detalhes, agindo como um assistente pessoal humano.
    Inclui a nova intenção para sugerir horário de estudo de IA.
    Retorna {'intent': 'erro_api', 'details': {}} se o modelo não foi carregado.
    """
    # Verifica se o modelo foi carregado antes de tentar usá-lo
    if model is None:
        # A mensagem de erro mais detalhada já foi exibida na configuração inicial.
        # Apenas retorna o erro para que o loop principal saiba o que fazer.
        return {"intent": "erro_api", "details": {}}

    # ** PROMPT PRINCIPAL AJUSTADO PARA PERSONA HUMANA E NOVAS INTENÇÕES **
    prompt = f"""
Você é meu assistente pessoal amigável, proativo e com um leve senso de humor. Sua tarefa é analisar a seguinte frase que eu, o usuário, disse e identificar a INTENÇÃO principal e extrair DETALHES relevantes para gerenciar meus itens (eventos, tarefas, aniversários), sugerir pausas, analisar meus itens ou sugerir nomes criativos, sugerir atividades/conteúdo baseado no meu humor, ou sugerir um horário para estudar IA.

Considere a data atual {datetime.now().strftime('%Y-%m-%d')} para inferir datas relativas como "hoje", "amanhã", "próxima segunda", "sexta-feira que vem", etc. Se o ano não for mencionado, assuma o ano atual.

As intenções possíveis são:
- adicionar_item: Eu quero adicionar um item (evento, tarefa ou aniversário) na minha lista.
- listar_itens: Eu quero ver meus itens.
- sugerir_pausa: Eu estou falando sobre precisar de uma pausa ou sugerir uma, com base nos meus itens.
- analisar_agenda: Eu quero uma análise ou resumo dos meus itens (dia, semana, etc.).
- renomear_item: Eu quero sugestões de nomes criativos para um item.
- sugerir_bem_estar_humor: Eu quero sugestões de atividades, filmes, livros ou comidas baseado no meu humor.
- **sugerir_estudo_ia**: Eu quero que você sugira um horário para eu estudar IA.
- sair: Eu quero encerrar a conversa.
- desconhecido: Se a intenção não for clara ou não se encaixar nas anteriores.

Para a intenção 'adicionar_item', extraia os seguintes detalhes:
- descricao: O título ou descrição do item.
- data: A data do item. Tente normalizar para o formatoYYYY-MM-DD. Se eu usar termos relativos, use a data atual para calcular. Se não houver data, use uma string vazia.
- hora: A hora de início do item. Tente normalizar para o formato HH:MM (24 horas). Se não houver hora, use uma string vazia. Tarefas e Aniversários podem não ter hora, neste caso use uma string vazia.
- categoria: Tente inferir uma categoria (Trabalho, Estudo, Pessoal, Bem-Estar, Lazer, Outro). Se não for claro, use 'Pessoal'.
- tipo_item: Identifique se o item é um "Evento", "Tarefa" ou "Aniversário". Se não for claro, use "Evento".

Para a intenção 'listar_itens', extraia:
- periodo: Se eu especificar um período (ex: "hoje", "amanhã", "esta semana", "próximo mês", "tudo"). Use termos como "dia", "semana", "mês", "tudo". Se não especificado, use "tudo".
- data: Se eu especificar uma data específica para listar (ex: "itens de segunda", "listar em 2025-06-01"). Normalize paraYYYY-MM-DD.

Para a intenção 'analisar_agenda', extraia:
- periodo: O período que eu quero analisar (ex: "hoje", "amanhã", "esta semana", "próximo mês"). Use termos como "dia", "semana", "mês", "tudo". Se não especificado, use "próximos itens".

Para a intenção 'renomear_item', extraia:
- descricao_original: A descrição atual do item que eu quero renomear.

Para a intenção 'sugerir_bem_estar_humor', extraia:
- humor: Como eu estou me sentindo (ex: "cansado", "feliz", "estressado", "entediado", "animado").
- tipo_sugestao: Que tipo de sugestão eu quero (ex: "atividades", "filmes", "livros", "comidas", "algo para fazer").

Para a intenção 'sugerir_estudo_ia', extraia:
- **confirmacao_inicial**: Se o usuário já está confirmando que quer a sugestão (ex: "sim, quero", "pode sugerir"). Use "sim" se a frase indicar confirmação, caso contrário "não".

Sua resposta DEVE ser APENAS um objeto JSON. Não inclua nenhum texto adicional antes ou depois do JSON. Não adicione explicações. Lembre-se de manter a persona amigável e com humor.

Examples:
Usuário: Adicionar evento "Reunião com chefe" amanhã às 10 da manhã.
Resposta JSON:
```json
{{
  "intent": "adicionar_item",
  "details": {{
    "descricao": "Reunião com chefe",
    "data": "YYYY-MM-DD (data de amanhã baseada na data atual)",
    "hora": "10:00",
    "categoria": "Trabalho",
    "tipo_item": "Evento"
  }}
}}
```

Usuário: Sugira um horário para estudar IA.
Resposta JSON:
```json
{{
  "intent": "sugerir_estudo_ia",
  "details": {{
    "confirmacao_inicial": "não"
  }}
}}
```
Usuário: Sim, pode sugerir um horário para estudar IA.
Resposta JSON:
```json
{{
  "intent": "sugerir_estudo_ia",
  "details": {{
    "confirmacao_inicial": "sim"
  }}
}}
```

Usuário: Estou me sentindo cansado, sugira um filme.
Resposta JSON:
```json
{{
  "intent": "sugerir_bem_estar_humor",
  "details": {{
    "humor": "cansado",
    "tipo_sugestao": "filme"
  }}
}}
```

Usuário: Sair
Resposta JSON:
```json
{{
  "intent": "sair",
  "details": {{}}
}}
```

Now, analyze the following phrase from the user:
"{comando_usuario}"

Sua resposta DEVE ser APENAS o JSON.
"""
    try:
        # Chama o modelo Gemini com o prompt. Temperatura 0 para extração estruturada
        response = model.generate_content(
            prompt,
            generation_config=genai.GenerationConfig(temperature=0.0)
        )

        json_text = response.text.strip()
        # Limpeza mais robusta de marcadores de código
        while json_text.startswith("```") or json_text.endswith("```"):
             if json_text.startswith("```json"):
                  json_text = json_text[7:].strip()
             elif json_text.startswith("```"):
                  json_text = json_text[3:].strip()
             if json_text.endswith("```"):
                  json_text = json_text[:-3].strip()


        parsed_response = json.loads(json_text)
        return parsed_response

    # Se der erro ao converter para JSON ou na chamada do Gemini, captura o erro
    except json.JSONDecodeError:
        print("Erro: Gemini não retornou um JSON válido para o comando. 😞 Que estranho!")
        # print(f"Resposta bruta do Gemini: {response.text}") # Descomente para ver a resposta bruta se houver erro
        return {"intent": "desconhecido", "details": {}} # Retorna desconhecido para que o bot não trave
    except Exception as e:
        print(f"Ocorreu um erro ao processar o comando com Gemini: {e} 😭 Ah, não!")
        return {"intent": "desconhecido", "details": {}} # Retorna desconhecido para que o bot não trava

# --- Loop Principal do Chatbot ---
# Este é o código que mantém o chatbot rodando e direciona as ações.

def iniciar_chatbot():
    """Inicia a interação com o usuário."""
    # Mensagem de boas-vindas com a nova persona
    print("✨ E aí! Eu sou seu Assistente Pessoal Inovador. ✨📅🧘‍♀️🧠")
    print("Pode me chamar de seu braço direito digital para organizar seus itens.")
    print("Vou além de apenas guardar seus compromissos! Posso dar uma olhada nos seus itens, sugerir pausas criativas, ajudar a dar nomes divertidos, sugerir atividades/conteúdo dependendo de como você está se sentindo, e até te ajudar a encontrar o melhor horário para estudar IA! 😉")
    print("\nO que posso fazer por você hoje? 🤔")
    print("- 'Adicionar [descrição] [data] [hora]' (Tipo: Evento, Tarefa, Aniversário. Ex: 'Adicionar reunião amanhã as 10h', 'Adicionar tarefa comprar pão hoje', 'Adicionar aniversário do João dia 15 de dezembro')")
    print("- 'Listar meus itens' (Para ver seus próximos compromissos, tarefas e aniversários. Sem fofoca, prometo! 😉)")
    print("- 'Sugira uma pausa' (Porque você merece um descanso! ☕️)")
    print("- 'Analisar meus itens [período]' (Ex: 'Analisar meus itens hoje', 'Analisar esta semana'. Vamos ver como as coisas estão!)")
    print("- 'Sugira nomes criativos para [descrição]' (Ex: 'Sugira nomes criativos para reunião chata'. Para dar um up na sua lista! 😂)")
    print("- 'Estou [humor], sugira [tipo]' (Ex: 'Estou cansado, sugira um filme', 'Quero algo para fazer, estou entediado'. Me diga como você está! 😊)")
    print("\nMe diga o que precisa! 😊")
    print("Digite 'sair' para me dar um tempinho. 👋")
    print("-" * 70) # Separador visual

    # VERIFICAÇÃO CRÍTICA: Se o modelo não carregou na configuração inicial, o chatbot não pode funcionar.
    if model is None:
        print("\nOps! Parece que não consegui me conectar com a inteligência artificial. 😟 Por favor, verifique os erros acima para me colocar de pé de novo!")
        return # Sai da função, impedindo o loop principal de rodar

    while True:
        comando_usuario = input("Você: ") # Espera a entrada do usuário

        # Verifica se o usuário quer sair (esta é a única parte que funciona sem o Gemini)
        if comando_usuario.lower() == 'sair':
            print("Saindo... Mas não suma por muito tempo! Lembre-se de cuidar do seu bem-estar e ser criativo com seus itens! 😉👋")
            break # Sai do loop, encerrando o chatbot

        # Entende o comando usando a função que chama o Gemini
        comando_analisado = entender_comando_com_gemini(comando_usuario)
        intent = comando_analisado.get("intent") # Pega a intenção identificada
        details = comando_analisado.get("details", {}) # Pega os detalhes extraídos

        # Imprime informações de debug (útil para ver o que o Gemini entendeu)
        print(f"\nDEBUG: Intenção identificada: {intent}")
        print(f"DEBUG: Detalhes extraídos: {details}")
        print("-" * 20)

        # --- Lógica para executar ações baseadas na intenção ---
        # Usa IF/ELIF para decidir o que fazer com base na intenção

        # Se a intenção for adicionar um item (Evento, Tarefa ou Aniversário)
        if intent == "adicionar_item":
            print("Ok, vamos adicionar isso na sua lista.")
            # Extrai os detalhes necessários do dicionário 'details'
            descricao = details.get('descricao', 'Novo Item')
            data = details.get('data', '')
            hora = details.get('hora', '') # Hora pode ser vazia para Tarefas/Aniversários
            categoria = details.get('categoria', 'Pessoal')
            tipo_item = details.get('tipo_item', 'Evento') # Pega o tipo de item extraído pelo Gemini

            # Validação básica: precisa de descrição e data. Hora é opcional dependendo do tipo.
            if descricao and data and data != "Sem data":
                 # Para Eventos, ainda é bom ter hora se possível
                 if tipo_item == "Evento" and not hora:
                     print("Hmm, para um Evento, geralmente precisamos da hora também. 😅 Poderia especificar? (Ex: 'Adicionar evento reunião amanhã as 10:00')")
                 else:
                     # Chama a função de adicionar, passando agora o tipo de item
                     adicionar_item(descricao, data, hora, categoria, tipo_item)
            else:
                 print("Ops! Não consegui pegar a **descrição e a data** do seu pedido. 🙁 Tente de novo? (Ex: 'Adicionar tarefa comprar pão hoje' ou 'Adicionar aniversário do João dia 15 de dezembro')")


        # Se a intenção for listar itens
        elif intent == "listar_itens":
            print("Claro! Dando uma espiadinha nos seus itens...")
            listar_itens() # Chama a função de listar

        # Se a intenção for sugerir pausa
        elif intent == "sugerir_pausa":
            print("Ah, pensando em um break? Boa! 🧘‍♀️ Deixa eu dar uma olhada nos seus itens para achar o momento perfeito...")
            # Reutiliza a função de análise, focando no contexto de pausas
            analisar_agenda_criativo(agenda_items, periodo="para sugerir pausas")

        # Se a intenção for analisar a agenda
        elif intent == "analisar_agenda":
            periodo_analise = details.get('periodo', 'próximos itens')
            print(f"Analisar seus itens para {periodo_analise}? Deixa comigo! 🕵️‍♀️")
            analisar_agenda_criativo(agenda_items, periodo=periodo_analise) # Chama a nova função de análise

        # Se a intenção for renomear item
        elif intent == "renomear_item":
             descricao_original = details.get('descricao_original', '')
             if descricao_original:
                 print(f"Renomear '{descricao_original}'? Adoro um desafio criativo! ✨")
                 renomear_item_criativo(descricao_original) # Chama a nova função de renomear
             else:
                 print("Hmm, qual item você quer deixar mais divertido? 🤔 Diga a descrição atual! (Ex: 'Sugira nomes criativos para reunião chata')")

        # --- Lógica: Sugerir Bem-Estar Baseado no Humor ---
        elif intent == "sugerir_bem_estar_humor":
             humor = details.get('humor', '')
             tipo_sugestao = details.get('tipo_sugestao', '')
             if humor and tipo_sugestao:
                 print(f"Entendido! Você está se sentindo {humor} e quer sugestões de {tipo_sugestao}. Deixa eu pensar em algo legal... 🤔")
                 sugerir_bem_estar_com_humor(humor, tipo_sugestao) # Chama a nova função de sugestão por humor
             else:
                 print("Para te dar sugestões personalizadas, preciso saber como você está se sentindo e o que você quer (atividades, filmes, livros, comidas). 😉 Tente algo como: 'Estou cansado, sugira um filme'.")
        # --- FIM DA LÓGICA ---

        # --- NOVA LÓGICA: Sugerir Horário para Estudar IA ---
        elif intent == "sugerir_estudo_ia":
            # A função sugerir_horario_estudo_ia já gerencia toda a interação (perguntar, sugerir, confirmar)
            sugerir_horario_estudo_ia(agenda_items)
        # --- FIM DA NOVA LÓGICA ---

        # Se a intenção for 'desconhecido' (Gemini não entendeu)
        elif intent == "desconhecido":
            print("Desculpe, não entendi bem o que você disse. 😅 Minha inteligência artificial às vezes precisa de uma ajudinha!")
            print("Posso ajudar com seus itens: adicionar (eventos, tarefas, aniversários), listar, sugerir pausas, analisar seus itens, sugerir nomes criativos, sugerir algo baseado no seu humor, ou sugerir um horário para estudar IA.")
            print("Tente algo como: 'Adicionar tarefa comprar leite amanhã', 'Listar meus itens', 'Sugira uma pausa', 'Analisar meus itens hoje', 'Estou feliz, sugira atividades', ou **'Sugira horário para estudar IA'**. 😉")


        elif intent == "erro_api":
             print("Ops! Tive um problema para me comunicar com a inteligência artificial agora. 😔 Por favor, tente de novo mais tarde!")
             # A mensagem de erro mais detalhada já foi exibida no início, esta é apenas um feedback durante a interação.

        # Para qualquer outra intenção que possa ter sido adicionada ao prompt mas não tratada aqui
        else:
            print(f"Hmm, essa intenção '{intent}' é nova para mim ({intent}). Ainda estou aprendendo! 🚧")

# --- Iniciar o Chatbot ---
# Esta linha inicia todo o processo chamando a função principal.
iniciar_chatbot()

Biblioteca google-generativeai instalada e API configurada. ✨ Prontos para conversar! 😉
Modelo 'gemini-2.0-flash' carregado com sucesso. 🎉 Vamos lá!
✨ E aí! Eu sou seu Assistente Pessoal Inovador. ✨📅🧘‍♀️🧠
Pode me chamar de seu braço direito digital para organizar seus itens.
Vou além de apenas guardar seus compromissos! Posso dar uma olhada nos seus itens, sugerir pausas criativas, ajudar a dar nomes divertidos, sugerir atividades/conteúdo dependendo de como você está se sentindo, e até te ajudar a encontrar o melhor horário para estudar IA! 😉

O que posso fazer por você hoje? 🤔
- 'Adicionar [descrição] [data] [hora]' (Tipo: Evento, Tarefa, Aniversário. Ex: 'Adicionar reunião amanhã as 10h', 'Adicionar tarefa comprar pão hoje', 'Adicionar aniversário do João dia 15 de dezembro')
- 'Listar meus itens' (Para ver seus próximos compromissos, tarefas e aniversários. Sem fofoca, prometo! 😉)
- 'Sugira uma pausa' (Porque você merece um descanso! ☕️)
- 'Analisar meus itens [período]' (Ex: 'Anal